# Seq2Seq model without using legacy stuff

In [3]:
! python -m spacy download en
! python -m spacy download de

Traceback (most recent call last):
  File "C:\Users\sdangayach162437\Anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py", line 485, in wrap_socket
    cnx.do_handshake()
  File "C:\Users\sdangayach162437\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 1934, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "C:\Users\sdangayach162437\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 1671, in _raise_ssl_error
    _raise_current_error()
  File "C:\Users\sdangayach162437\Anaconda3\lib\site-packages\OpenSSL\_util.py", line 54, in exception_from_error_queue
    raise exception_type(errors)
OpenSSL.SSL.Error: [('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\sdangayach162437\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "C:\Users\sdangayach162437\Anaconda3\l

In [2]:
!pip install torchtext

In [3]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List

import spacy

In [4]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

In [5]:
# Create source and target language tokenizer. Make sure to install the dependencies.
token_transform_de = get_tokenizer('spacy', language='de')
token_transform_en = get_tokenizer('spacy', language='en')

In [6]:
# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    if language == SRC_LANGUAGE:
        for src, _ in data_iter:
            yield token_transform_de(src)
    else:
        for _ , tgt in data_iter:
            yield token_transform_en(tgt)

In [7]:
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [8]:
# Training and Testing data Iterator 
train_iter, test_iter = Multi30k(split=('train', 'test'), language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

In [9]:
# check the split size
len(train_iter), len(test_iter)

(29000, 1000)

In [10]:
# test iterator
next(train_iter)

('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.\n',
 'Two young, White males are outside near many bushes.\n')

In [11]:
# Create torchtext's train Vocab object 
vocab_transform_trn_src = build_vocab_from_iterator(yield_tokens(train_iter, SRC_LANGUAGE),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
vocab_transform_trn_src.set_default_index(UNK_IDX)

vocab_transform_trn_tgt = build_vocab_from_iterator(yield_tokens(train_iter, TGT_LANGUAGE),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
vocab_transform_trn_tgt.set_default_index(UNK_IDX)

In [12]:
# Test Vocab Transform
# vocab_transform_trn_tgt([('Hallo, mein Name ist Sachin','Hi my name is sachin')])

In [15]:
a = yield_tokens([('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.\n','Hi my name is sachin')], SRC_LANGUAGE)

In [16]:
[i for i in a]

[['Zwei',
  'junge',
  'weiße',
  'Männer',
  'sind',
  'im',
  'Freien',
  'in',
  'der',
  'Nähe',
  'vieler',
  'Büsche',
  '.',
  '\n']]

In [ ]:
vocab_transform_trn_src([('Zwei junge','hi')])